### Problem Statement


In [ ]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, clean_llm_output, load_artifact, render_plantuml_diagram

# Initialize the LLM client. You can change the model here.
# For example: setup_llm_client(model_name="gemini-2.5-flash")
client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

In [ ]:
## Generate the PRD

problem_statement = f"""
We want to write a simple application that connects users to vacation rentals. 
The application should allow users to get a recommendation for vacation rentals based on 
their desired activities and/or attractions they wish to visit.

Users will have the ability to make a reservation at their selected property.

"""

prd_prompt = f"""# Generate a Product Requirements Document (PRD) for the following problem statement:
{problem_statement} 
# The PRD should include the following sections:
1. **Problem Statement**: A clear and concise description of the problem.  
2. **Objectives**: The goals we want to achieve with this application.
3. **User Stories**: A list of user stories that describe the features from the user's perspective.
4. **Acceptance Criteria**: The criteria that must be met for the user stories to be considered complete.
5. **Out of Scope**: Features that are not included in the initial release.

 # We are focused on the initial MVP (Minimum Viable Product) and will include only the essential features.
    1. User can specify a list of activities or attractions they want to visit.
    2. User can get a recommendation for vacation rentals based on their desired activities or attractions
    3. User can make a reservation at their selected property.

# The PRD should be in Markdown format and should be well-structured.
  The PRD should be concise and to the point, avoiding unnecessary details.
    
  """  

print("--- Generating  PRD ---")

prd_output = get_completion(prd_prompt, client, model_name, api_provider)
prd_output_clean = clean_llm_output(prd_output, "markdown")
save_artifact( prd_output_clean, "capstone_artifacts/prd.md",)



In [ ]:

# Load the PRD content for schema generation
prd_content = load_artifact("capstone_artifacts/prd.md")
schema_prompt = f"""
You are an expert Database Administrator.
Using this PRD as a reference: {prd_content}
From this PRD, create a normalized SQL schema with only the following three tables:
1. **Users**: To store user information and their interests
2. **Properties**: To store vacation rental properties located in the US.  Include fields for full address.
        No need for latititude/longitude info.
3. **Reservations**: To store user reservations for properties.
The output should be the raw `CREATE TABLE` statements.  SQL should be compatible with sqllite3.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'capstone_artifacts/schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""

In [ ]:
cleaned_schema = load_artifact('capstone_artifacts/schema.sql')
prd_content = load_artifact('capstone_artifacts/prd.md')
# TODO: Write a prompt to generate realistic seed data.

seed_data_prompt = f"""You are a senior-level Database Engineer with deep expertise in data modeling, SQL, and realistic data generation.

Use the following Product Requirements Document (PRD) as domain context: {prd_content}

Use the following SQLite-compatible schema as the structural reference: {cleaned_schema}

Your task is to generate seed data for a vacation rental system. The output should consist of pure SQL `INSERT INTO` statements, compatible with SQLite3, with no explanation or additional formatting.

Generate seed data for the following tables:

1. **Users**
   - Insert at least 10 realistic users.
   - Include a variety of demographics (age, gender, region) and interests (e.g., hiking, food, culture).
   - Make sure the data reflects diverse travel preferences.

2. **Properties**
   - Insert at least 100 vacation rental properties.
   - Each property should include a full US address (street, city, state, zip).
   - Include amenities such as Wi-Fi, pool, pet-friendly, etc.
   - Distribute properties across a mix of major cities (e.g., NYC, LA, Chicago) and rural destinations (e.g., Sedona, Lake Tahoe, Smoky Mountains).

3. **Reservations**
   - Insert at least 5 sample reservations.
   - Link existing users to properties with reasonable travel dates.
   - Avoid date conflicts or duplicates.
   - Include a mix of short and longer stays, seasonal variation, and different users.

Ensure all foreign key relationships are valid, and all timestamps and locations make sense in context.

Return only the SQL seed statements — no commentary, no markdown, no code fencing.
"""


print("--- Generating Seed Data ---")
if prd_content and cleaned_schema:
    generated_seed_data = get_completion(seed_data_prompt, client, model_name, api_provider)
    
    # Clean up the generated seed data
    cleaned_seed_data = clean_llm_output(generated_seed_data, language='sql')
    print(cleaned_seed_data)
    
    # Save the cleaned seed data
    save_artifact(cleaned_seed_data, 'capstone_artifacts/seed_data.sql')
else:
    print("Skipping seed data generation because PRD or schema is missing.")

In [ ]:
schema = load_artifact('capstone_artifacts/schema.sql')
prd_content = load_artifact('capstone_artifacts/prd.md')

arch_doc_prompt = f"""
You are a seasoned Solution Architect with expertise in designing scalable, modular web applications.

Your task is to generate a well-structured, high-level **Architecture Document** in Markdown format for a vacation rental platform. Base your design on:

- Product Requirements Document (PRD): {prd_content}
- Database Schema (SQLite-compatible): {cleaned_schema}

Your document should include the following sections:

---

1. **Overview**
   - Provide a brief summary of the system’s purpose, primary user personas, and key business goals.
   - Highlight any constraints (tech stack, budget, timeline) or assumptions.

2. **User Interface**
   - Describe the main UI components and user flows: searching rentals, filtering by interests, making reservations.
   - Specify technologies: React + Tailwind CSS.
   - Mention responsiveness, accessibility, and interaction patterns (e.g. modals, forms, map integrations).

3. **Backend Services**
   - Detail API endpoints and service responsibilities: user authentication, reservation logic, property search.
   - Note use of Python with FastAPI.
   - Explain service modularity, request routing, data validation, and async processing (if applicable).

4. **Database Design**
   - Summarize the schema structure: tables, relationships, constraints, indexes.
   - Address how user, property, and reservation data are stored and queried.
   - Call out any SQLite-specific design considerations (e.g. lack of native concurrency or full-text search).

5. **Property Recommendation Engine**
   - Explain how user interests are captured and interpreted.
   - Describe how an LLM will be used to match users to relevant properties.
   - Include input/output formats, example prompt flow, and scoring or filtering strategies.

6. **Deployment & DevOps (Optional)**
   - Mention local development setup, environment configuration, and deployment pipeline (if relevant).
   - Could include use of Docker, CI/CD tools, or cloud platforms.

---

Output only the Markdown document — no extra commentary or formatting. Ensure it reads clearly for technical and non-technical stakeholders.


"""    

print("--- Generating Architecture Doc ---")
if schema and prd_content:
    generated_arch_doc = get_completion(arch_doc_prompt, client, model_name, api_provider)
    
    # Clean up the generated architecture document
    cleaned_arch_doc = clean_llm_output(generated_arch_doc, "markdown")
    
    # Save the cleaned seed data
    save_artifact(cleaned_arch_doc, 'capstone_artifacts/architecture_doc.md')
else:
    print("Skipping architecture doce generation because PRD or schema is missing.")

In [ ]:
architecture_doc = load_artifact('capstone_artifacts/architecture_doc.md')

# Generate a component diagram for the architecture document
component_diagram_prompt = """
You are an expert system architect. Generate PlantUML code for a component diagram that describes 
a web application architecture for a vacation rental platform based on the following requirements:

The diagram should include the following four components:
- A 'Frontend' (e.g., a web browser)
- A 'Backend API Server'
- A 'LLM-based Recommendation Engine'
- A 'Database'

The relationships are as follows:
- The Frontend communicates with the Backend API Server over HTTPS for tasks like view assigned tasks for users and create tasks for admins.
- The Backend API Server reads from and writes to the Database.
- The Backend API Server sends requests to the LLM-based Recommendation Engine to get property 
    recommendations based on user interests.

Do not generate or include any custom classes in the output.
Output only the raw PlantUML code inside a markdown block.
"""

print("--- Generating Component Diagram ---")
component_puml_raw = get_completion(component_diagram_prompt, client, model_name, api_provider)
component_puml = clean_llm_output(component_puml_raw, language='plantuml')

print("\n--- Generated PlantUML Code ---")
print(component_puml)
if component_puml:
    save_artifact(component_puml, 'capstone_artifacts/app_component_diagram.puml')

# Render the diagram
if component_puml:
    render_plantuml_diagram(component_puml, "capstone_artifacts/app_component_diagram.png")




    